## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [3]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df=pd.read_csv('/content/train.csv', on_bad_lines='skip', engine='python', encoding='latin-1')

In [6]:
df.shape

(20900, 5)

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df.isnull().sum()

,0
id,0
title,582
author,2016
text,112
label,85


In [9]:
###Drop Nan Values
df=df.dropna()


In [10]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [12]:
## Get the Dependent features
# Filter out rows where 'label' is not a digit
df_filtered = df[df['label'].astype(str).str.isdigit()]
y = df_filtered['label'].astype(int)
X = df_filtered.drop('label', axis=1) # Also update X to match the filtered rows

In [13]:
X.shape

(18283, 4)

In [14]:
y.shape

(18283,)

In [15]:
import tensorflow as tf

In [16]:
tf.__version__

'2.18.0'

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [18]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [19]:
messages=X.copy()

In [20]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [21]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20895,20795,Rapper T.I.: Trump a âPoster Child For White...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20896,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20897,20797,Macyâs Is Said to Receive Takeover Approach ...,Michael J. de la Merced and Rachel Abrams,The Macyâs of today grew from the union of s...
20898,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [22]:
messages.reset_index(inplace=True)

In [23]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18278,20895,20795,Rapper T.I.: Trump a âPoster Child For White...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18279,20896,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18280,20897,20797,Macyâs Is Said to Receive Takeover Approach ...,Michael J. de la Merced and Rachel Abrams,The Macyâs of today grew from the union of s...
18281,20898,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [24]:
import nltk
import re
from nltk.corpus import stopwords

In [25]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [27]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [28]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [29]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[310, 4126, 1449, 1067, 2140, 496, 359, 1935, 4308, 2707],
 [2234, 2292, 4246, 2703, 4237, 2530, 3384],
 [1398, 13, 2969, 4433],
 [116, 771, 2863, 1624, 3535, 1413],
 [1806, 4237, 3467, 2024, 2987, 3531, 4237, 3756, 3401, 108],
 [4554,
  4988,
  1895,
  1840,
  2694,
  2230,
  2290,
  331,
  904,
  2381,
  790,
  2581,
  695,
  2651,
  3384],
 [2330, 1401, 470, 660, 4386, 229, 1887, 2278, 4920, 605, 2580],
 [4863, 2335, 1494, 1274, 672, 932, 2230, 1616, 4920, 605, 2580],
 [117, 1193, 2132, 3137, 3848, 1592, 1981, 4693, 2230, 1395],
 [49, 1893, 1003, 3434, 1433, 4466, 2883, 1456],
 [2221, 2349, 324, 4817, 4907, 4363, 1137, 307, 2547, 388, 2863],
 [1624, 1945, 2140, 1592, 2230, 672],
 [3933, 1581, 1198, 1714, 964, 3584, 233, 4356, 3753],
 [2742, 25, 156, 3103, 3880, 2942, 2774, 4920, 605, 2580],
 [3313, 1032, 216, 1752, 2441, 4920, 605, 2580],
 [2858, 2578, 3509, 4815, 453, 599, 3215, 1012, 2306, 1782],
 [81, 735, 2292],
 [3705, 4901, 2305, 1662, 2230, 1845, 4084, 3384],
 [685, 348, 424

In [30]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [31]:
onehot_repr[1]

[2234, 2292, 4246, 2703, 4237, 2530, 3384]

### Embedding Representation

In [32]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[ 310 4126 1449 ...    0    0    0]
 [2234 2292 4246 ...    0    0    0]
 [1398   13 2969 ...    0    0    0]
 ...
 [4081 2974 1429 ...    0    0    0]
 [1870  672 4334 ...    0    0    0]
 [ 810 4197 4343 ...    0    0    0]]


In [33]:
embedded_docs[1]

array([2234, 2292, 4246, 2703, 4237, 2530, 3384,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [34]:
embedded_docs[0]

array([ 310, 4126, 1449, 1067, 2140,  496,  359, 1935, 4308, 2707,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [35]:
## Creating model
embedding_vector_features=40 ##features representation #This is hyper parameter
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [36]:
len(embedded_docs),y.shape

(18283, (18283,))

In [37]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y, dtype=int)

In [38]:
X_final.shape,y_final.shape

((18283, 20), (18283,))

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [40]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.7596 - loss: 0.4312 - val_accuracy: 0.9151 - val_loss: 0.2056
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.9525 - loss: 0.1318 - val_accuracy: 0.9193 - val_loss: 0.2052
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.9741 - loss: 0.0817 - val_accuracy: 0.9191 - val_loss: 0.2240
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 42ms/step - accuracy: 0.9820 - loss: 0.0567 - val_accuracy: 0.9143 - val_loss: 0.2563
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - accuracy: 0.9871 - loss: 0.0385 - val_accuracy: 0.9166 - val_loss: 0.2713
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.9889 - loss: 0.0339 - val_accuracy: 0.9160 - val_loss: 0.3391
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.9932 - loss: 0.0232 - val_accuracy: 0.9110 - val_loss: 0.4111
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.9944 - loss: 0.0193 - val_

### Adding Dropout

In [41]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features)) ##,input_length=sent_length
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [42]:
### Again Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.7578 - loss: 0.4336 - val_accuracy: 0.9211 - val_loss: 0.1985
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 15s 62ms/step - accuracy: 0.9485 - loss: 0.1361 - val_accuracy: 0.9195 - val_loss: 0.1906
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.9670 - loss: 0.0975 - val_accuracy: 0.9185 - val_loss: 0.2048
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.9734 - loss: 0.0832 - val_accuracy: 0.9155 - val_loss: 0.2626
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.9793 - loss: 0.0651 - val_accuracy: 0.9175 - val_loss: 0.2245
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.9864 - loss: 0.0496 - val_accuracy: 0.9176 - val_loss: 0.2412
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.9861 - loss: 0.0488 - val_accuracy: 0.9147 - val_loss: 0.2968
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.9885 - loss: 0.0404 - val

In [43]:
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 40)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769,505 (2.94 MB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 513,004 (1.96 MB)

None


### Performance Metrics And Accuracy

In [44]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [45]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [46]:
from sklearn.metrics import confusion_matrix

In [47]:
confusion_matrix(y_test,y_pred)

array([[3104,  311],
       [ 228, 2391]])

In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9106728538283063

In [49]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3415
           1       0.88      0.91      0.90      2619

    accuracy                           0.91      6034
   macro avg       0.91      0.91      0.91      6034
weighted avg       0.91      0.91      0.91      6034

